In [2]:
%matplotlib inline
import pandas as pd
import scipy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches

In [3]:
custom_style = {
    'figure.figsize': (8, 6),
    'font.family':'sans-serif',
    'axes.facecolor': 'lightgray',
    'figure.facecolor':'silver',
    'scatter.marker': 'o',
    'font.size': 9,
    'axes.grid': True
    }

In [11]:
def soma():
    x=float(input("Número:"))
    y=float(input("Número:"))
    print(x+y)
    return x

soma()

4.0


2.0

#### Importação de dados (input)

In [6]:
def impt():
    df1 = pd.read_csv("Poços_Marinho/1-ALS-2-AL.csv")
    df=df1[["Profundidade","COT","S1","S2","S3","Tmax","IH","IO"]]
    return df

impt()

,Profundidade,COT,S1,S2,S3,Tmax,IH,IO
0,579.8,2.50,0.32,6.85,1.22,430.0,274.00,48.80
1,597.8,2.51,0.29,7.13,1.27,429.0,284.06,50.60
2,615.8,2.18,0.22,6.05,1.26,430.0,277.52,57.80
3,633.8,2.06,0.26,5.28,1.10,430.0,256.31,53.40
4,651.8,1.29,0.17,1.76,1.36,428.0,136.43,105.43
...,...,...,...,...,...,...,...,...
131,3285.8,0.62,NaN,NaN,NaN,NaN,NaN,NaN
132,3288.8,0.72,0.20,0.33,0.19,443.0,45.83,26.39
133,3303.8,0.26,0.10,0.10,0.30,NaN,38.46,115.38
134,3336.8,0.72,0.20,0.20,0.20,NaN,27.78,27.78


##### Cálculo de IH0ativo
* IHa = 100 x (S2/(COTa))

In [8]:
def IHa():
    impt()
    IHa = (100*df["S2"]/df["COT"])
    IHa_cor=[]
    for i in IHa:
        if i<0:
            IHa_cor.append(0)
        elif i>999:
            IHa_cor.append(999)
        else:
            IHa_cor.append(i) # Eliminar valores
    df["IH0a"]= IHa_cor
    return df
IHa()

NameError: name 'df' is not defined

##### Calculo da Taxa de Transformação sem Ro (IH)
* (IHa - IH)/ IHa

In [ ]:
def Tr_IH():
    df["Taxa de Transformação (IH)"] = ((df["IH0a"]-df["IH"])/df["IH0a"])
    TrIH=[]
    for i in df["Taxa de Transformação (IH)"]:
        if i<0:
            TrIH.append(0)
        elif i>1:
            TrIH.append=1
        else:
            TrIH.append(i)
    df["Taxa de Transformação (IH)"]=TrIH
    print(df)
Tr_IH()

     Profundidade   COT    S1    S2    S3   Tmax      IH      IO        IH0a  \
0           579.8  2.50  0.32  6.85  1.22  430.0  274.00   48.80  274.000000   
1           597.8  2.51  0.29  7.13  1.27  429.0  284.06   50.60  284.063745   
2           615.8  2.18  0.22  6.05  1.26  430.0  277.52   57.80  277.522936   
3           633.8  2.06  0.26  5.28  1.10  430.0  256.31   53.40  256.310680   
4           651.8  1.29  0.17  1.76  1.36  428.0  136.43  105.43  136.434109   
..            ...   ...   ...   ...   ...    ...     ...     ...         ...   
131        3285.8  0.62   NaN   NaN   NaN    NaN     NaN     NaN         NaN   
132        3288.8  0.72  0.20  0.33  0.19  443.0   45.83   26.39   45.833333   
133        3303.8  0.26  0.10  0.10  0.30    NaN   38.46  115.38   38.461538   
134        3336.8  0.72  0.20  0.20  0.20    NaN   27.78   27.78   27.777778   
135        3369.8  0.52   NaN   NaN   NaN    NaN     NaN     NaN         NaN   

     Taxa de Transformação (IH)  Ro (ca

##### Cálculo de Ro a partir de Tmax
* Ro calculada = (0,0180 x Tmax) - 7,16

In [ ]:
def Ro_cal():
    df["Ro (calculada)"] = (0.0180*df["Tmax"])-7.16
    Roc_cor=[]
    for i in df["Ro (calculada)"]:
        if i<0:
            Roc_cor.append(0)
        else:
            Roc_cor.append(i)
    df["Ro (calculada)"]=Roc_cor
    print(df)
Ro_cal()

     Profundidade   COT    S1    S2    S3   Tmax      IH      IO        IH0a  \
0           579.8  2.50  0.32  6.85  1.22  430.0  274.00   48.80  274.000000   
1           597.8  2.51  0.29  7.13  1.27  429.0  284.06   50.60  284.063745   
2           615.8  2.18  0.22  6.05  1.26  430.0  277.52   57.80  277.522936   
3           633.8  2.06  0.26  5.28  1.10  430.0  256.31   53.40  256.310680   
4           651.8  1.29  0.17  1.76  1.36  428.0  136.43  105.43  136.434109   
..            ...   ...   ...   ...   ...    ...     ...     ...         ...   
131        3285.8  0.62   NaN   NaN   NaN    NaN     NaN     NaN         NaN   
132        3288.8  0.72  0.20  0.33  0.19  443.0   45.83   26.39   45.833333   
133        3303.8  0.26  0.10  0.10  0.30    NaN   38.46  115.38   38.461538   
134        3336.8  0.72  0.20  0.20  0.20    NaN   27.78   27.78   27.777778   
135        3369.8  0.52   NaN   NaN   NaN    NaN     NaN     NaN         NaN   

     Taxa de Transformação (IH)  Ro (ca

#### Classificação do querogênio
* Tipo I: {IH > 600} ___ {S1/S3 > 15}
* Tipo II: {IH > 300 and IH <= 600} ___ {S1/S3 > 10 and S1/S3 <15}
* Tipo II/III: {IH > 200 and IH <=300} ___ {S1/S3 > 5 and S1/S3 < 10]
* Tipo III: {IH > 50 and IH <= 200} ___ {S1/S3 > 1 and S1/S3 < 5}
* Tipo IV: {IH <= 50} ___ {S1/S3 > 1}

In [ ]:
def K_Type():
    K = []
    for i in df["IH"]:
        if i > 600:
            a="I"
        elif i > 300 and i <=600:
            a="II"
        elif i > 200 and i <= 300:
            a="II/III"
        elif i > 50 and i <= 200:
            a="III"
        elif i>0 and i<=50:
            a="IV"
        else:
            a=""
        K.append(a)
    querogênio=pd.DataFrame(K)
    df["Querogênio"]=querogênio
    print(df)
K_Type()


     Profundidade   COT    S1    S2    S3   Tmax      IH      IO        IH0a  \
0           579.8  2.50  0.32  6.85  1.22  430.0  274.00   48.80  274.000000   
1           597.8  2.51  0.29  7.13  1.27  429.0  284.06   50.60  284.063745   
2           615.8  2.18  0.22  6.05  1.26  430.0  277.52   57.80  277.522936   
3           633.8  2.06  0.26  5.28  1.10  430.0  256.31   53.40  256.310680   
4           651.8  1.29  0.17  1.76  1.36  428.0  136.43  105.43  136.434109   
..            ...   ...   ...   ...   ...    ...     ...     ...         ...   
131        3285.8  0.62   NaN   NaN   NaN    NaN     NaN     NaN         NaN   
132        3288.8  0.72  0.20  0.33  0.19  443.0   45.83   26.39   45.833333   
133        3303.8  0.26  0.10  0.10  0.30    NaN   38.46  115.38   38.461538   
134        3336.8  0.72  0.20  0.20  0.20    NaN   27.78   27.78   27.777778   
135        3369.8  0.52   NaN   NaN   NaN    NaN     NaN     NaN         NaN   

     Taxa de Transformação (IH)  Ro (ca

#### End Member para tipo de querogênio

In [ ]:
# End Member
def k_fuzzy():
    T1 = []
    T2 = []
    T3 = []
    T4 = []
    for i in df["IH"]: 
        if i >= 800:
            end = ((i - 999)/(800 - 999))*100
            T1.append(100) 
            T2.append(0)
            T3.append(0)
            T4.append(0) 
        elif i >= 425 and i <=800:
            end = ((i - 800)/(425 - 800))*100
            T1.append(100-end) 
            T2.append(end)
            T3.append(0)
            T4.append(0) 
        elif i >= 150 and i < 425:
            end = ((i - 425)/(150 - 425))*100
            T1.append(0) 
            T2.append(100-end)
            T3.append(end)
            T4.append(0) 
        elif i >=25 and i < 150:
            end = ((i - 150)/(25 - 150))*100
            T1.append(0) 
            T2.append(0)
            T3.append(100-end)
            T4.append(end) 
        elif i>=0 and i < 25:
            T1.append(0) 
            T2.append(0)
            T3.append(0)
            T4.append(100) 
        else:
            T1.append(0) 
            T2.append(0)
            T3.append(0)
            T4.append(0) 
    df["% Tipo I"]=T1
    df["% Tipo II"]=T2
    df["% Tipo III"]=T3
    df["% Tipo IV"]=T4
    df.to_csv("Planilha.csv")
    print(df)
k_fuzzy()





     Profundidade   COT    S1    S2    S3   Tmax      IH      IO        IH0a  \
0           579.8  2.50  0.32  6.85  1.22  430.0  274.00   48.80  274.000000   
1           597.8  2.51  0.29  7.13  1.27  429.0  284.06   50.60  284.063745   
2           615.8  2.18  0.22  6.05  1.26  430.0  277.52   57.80  277.522936   
3           633.8  2.06  0.26  5.28  1.10  430.0  256.31   53.40  256.310680   
4           651.8  1.29  0.17  1.76  1.36  428.0  136.43  105.43  136.434109   
..            ...   ...   ...   ...   ...    ...     ...     ...         ...   
131        3285.8  0.62   NaN   NaN   NaN    NaN     NaN     NaN         NaN   
132        3288.8  0.72  0.20  0.33  0.19  443.0   45.83   26.39   45.833333   
133        3303.8  0.26  0.10  0.10  0.30    NaN   38.46  115.38   38.461538   
134        3336.8  0.72  0.20  0.20  0.20    NaN   27.78   27.78   27.777778   
135        3369.8  0.52   NaN   NaN   NaN    NaN     NaN     NaN         NaN   

     Taxa de Transformação (IH)  Ro (ca

#### Decisão sobre tipo de Querogênio

In [ ]:
x=['% Tipo I', '% Tipo II', '% Tipo III', '% Tipo IV']
def k_sum():
    TI=df["% Tipo I"].dropna().values.sum()
    TII=df["% Tipo II"].dropna().values.sum()
    TIII=df["% Tipo III"].dropna().values.sum()
    TIV=df["% Tipo IV"].dropna().values.sum()
k_sum(TI)
data=[TI,TII,TIII,TIV]
print(data)

NameError: name 'TI' is not defined

In [ ]:
def k_barplot():
    k_sum()
    plt.bar(x,data,0.4, facecolor="dimgray")
    #custom_labels_x = ['% Tipo I', '% Tipo II', '% Tipo III', '% Tipo IV']
    #plt.xticks(np.arange(1, len(custom_labels_x) + 1), custom_labels_x)
    plt.style.use(custom_style)
    plt.ylabel("% Tipo de Querogênio")
    plt.savefig("Barplot_K.pdf")
    plt.show()
k_barplot()

[609.584, 2500.205090909091, 4670.314909090908, 1719.8960000000002]


NameError: name 'x' is not defined

#### Cálculo de Tr por Ro e tipo de querogênio
* Tipo_I = -34.430609 + (183.63837 * Ro) - (361.494 * Ro**2) + (309.9 * Ro**3) - (96.8 * Ro**4)
* Tipo_II = -822.70308 + (6217.2684 * Ro) - (19265.314 * Ro**2) + (31326.872 * Ro**3) - (28204.703 * Ro**4) + (13345.477 * Ro**5) - (2595.9299 * Ro**6)
* Tipo_III = 6.6516023 - (33.879196 * Ro) + (64.978399 * Ro**2) - (60.264818 * Ro**3) + (29.700408 * Ro**4) - (7.5019085 * Ro**5) + (0.7656397 * Ro**6)


In [ ]:
def k_Tr():
    Ro = df["Ro (calculada)"].mean()
    print("Valor medio da Ro calculada:", Ro )
    #for Ro in df["Ro (calculada)"]:
    Eq_I = -34.430609 + (183.63837 * Ro) - (361.494 * Ro**2) + (309.9 * Ro**3) - (96.8 * Ro**4)
    Eq_II = -822.70308 + (6217.2684 * Ro) - (19265.314 * Ro**2) + (31326.872 * Ro**3) - (28204.703 * Ro**4) + (13345.477 * Ro**5) - (2595.9299 * Ro**6)
    Eq_III = 6.6516023 - (33.879196 * Ro) + (64.978399 * Ro**2) - (60.264818 * Ro**3) + (29.700408 * Ro**4) - (7.5019085 * Ro**5) + (0.7656397 * Ro**6)
        
    if TI > TII and TI > TIII and TI > TIV:
        TRo=Eq_I
        print("Querogênio Tipo I")
    elif TII > TI and TII > TIII and TII > TIV:
        TRo=Eq_II
        print("Querogênio Tipo II")
    elif TIII > TI and TIII > TII and TIII > TIV:
        TRo=Eq_III
        print("Querogênio Tipo III")
    else:
        TRo=0
        print("Querogênio Tipo IV")
    print("Taxa de TRansformação (Ro):", TRo)

    if i > 1:
        TRo = 1
    elif TRo < 0:
        TRo = 0
    else:
        TRo = TRo
k_Tr()

Valor medio da Ro calculada: 0.6936129032258057


NameError: name 'TI' is not defined